## Implementare misure di similarità basate su WordNet e calcolare indici di correlazione

In [1]:
#Import e costanti
import csv
import math

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from nltk.corpus import wordnet

WN_3_1_MAX_DEPTH=20

In [2]:
#Utilities
def read_csv(filename):
    dataset=[]
    with open(filename, 'r') as file:
        # Create a CSV reader object
        reader = csv.reader(file)
        for row in reader:
            row[2]=float(row[2])
            dataset.append(row+[None,None,None])
    return dataset

def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# Misura di similarità di Wu & Palmer

In [3]:
def calculate_wu_palmer(dataset):
    for row in dataset:
        #Le righe del dataset sono array di due elementi
        #Estraggo i synset dei termini
        first_elem_synset=wordnet.synsets(row[0])
        second_elem_synset = wordnet.synsets(row[1])
        #Costante per salvare la massima similarità tra elementi
        max=-1
        #Provo tutte le combinazioni di synset
        for first_synset in first_elem_synset:
            for second_synset in second_elem_synset:
                first_elem_height=first_synset.max_depth()+1
                second_elem_height=second_synset.max_depth()+1
                break_loop=True
                #While per la ricerca del del Lowest Common Subsumer
                #Uso una variabile loop perchè per alcune coppie si rischia di non trovare il valore quindi gestiamo la casistica
                while(break_loop):
                    if(first_synset == second_synset):
                     break_loop=False
                    #Se non abbiamo trovato il LCS faccio salire il synset più profondo
                    elif(first_synset.max_depth() > second_synset.max_depth()):
                        hyper=first_synset.hypernyms()
                        if(len(hyper)>0):
                            first_synset = hyper[0]
                        else:
                            break_loop=False
                            first_elem_height=-1
                    else:
                        hyper = second_synset.hypernyms()
                        if (len(hyper) > 0):
                            second_synset = hyper[0]
                        else:
                            break_loop = False
                            second_elem_height = -1
                lcs_height=first_synset.max_depth()+1
                #Se l'LCS è stato trovato
                if(first_elem_height>=0 and second_elem_height>=0):
                    index=((2*lcs_height)/(first_elem_height+second_elem_height))*10
                else:
                    index=-1
                #Index rappresenta la similarità, in caso sia il nuovo massimo lo aggiorno
                if(index>max):
                    max=index
        #I massimi per ogni coppia vengono inseriti nella terza cella in modo da creare un dataset con tutte le similarità
        row[3]=max
    return dataset

# Shortest Path

In [4]:
def calculate_shortest_path(dataset):
    for row in dataset:
        #Le righe del dataset sono array di due elementi
        #Estraggo i synset dei termini
        first_elem_synset=wordnet.synsets(row[0])
        second_elem_synset = wordnet.synsets(row[1])
        max_depth = WN_3_1_MAX_DEPTH
        #Costante per salvare la massima similarità tra elementi
        max=-1
        #Provo tutte le combinazioni di synset
        for first_synset in first_elem_synset:
            for second_synset in second_elem_synset:
                #Steps rappresenta il numero di passi nel cammino da un synset all'altro
                steps=0
                #Uso while simile a ricerca del LCS segnando però il numero di hypernyms attraversato
                #Uso una variabile loop perchè per alcune coppie si rischia di non trovare il valore quindi gestiamo la casistica
                break_loop=True
                while(break_loop):
                    if(first_synset == second_synset):
                     break_loop=False
                    elif(first_synset.max_depth() > second_synset.max_depth()):
                        hyper=first_synset.hypernyms()
                        if(len(hyper)>0):
                            first_synset = hyper[0]
                        else:
                            break_loop=False
                            steps=-2
                    else:
                        hyper = second_synset.hypernyms()
                        if (len(hyper) > 0):
                            second_synset = hyper[0]
                        else:
                            break_loop = False
                            steps = -2
                    steps+=1
                #Se è stato trovato un cammino
                if(steps>=0):
                    #Normalizzo per il valore massimo e moltiplico per 10 per avere un valore nel range [1,10]
                    index=index=normalize(((2*max_depth)-steps),0,(2*max_depth))*10
                else:
                    index=-1
                 #Index rappresenta la similarità, in caso sia il nuovo massimo lo aggiorno
                if(index>max):
                    max=index
        #I massimi per ogni coppia vengono inseriti nella quarta cella in modo da creare un dataset con tutte le similarità
        row[4]=max
    return dataset

# Leakcock & Chodorow

In [5]:
def calculate_leakcock_chodorow(dataset):
    for row in dataset:
        #Le righe del dataset sono array di due elementi
        #Estraggo i synset dei termini
        first_elem_synset=wordnet.synsets(row[0])
        second_elem_synset = wordnet.synsets(row[1])
        max_depth = WN_3_1_MAX_DEPTH
        #Costante per salvare la massima similarità tra elementi
        max = -1
        #Provo tutte le combinazioni di synset
        for first_synset in first_elem_synset:
            for second_synset in second_elem_synset:
                #Steps rappresenta il numero di passi nel cammino da un synset all'altro
                steps=0
                break_loop=True
                #Uso while come in shortest path per trovare la lunghezza del cammino
                #Uso una variabile loop perchè per alcune coppie si rischia di non trovare il valore quindi gestiamo la casistica
                while(break_loop):
                    if(first_synset == second_synset):
                     break_loop=False
                    elif(first_synset.max_depth() > second_synset.max_depth()):
                        hyper=first_synset.hypernyms()
                        if(len(hyper)>0):
                            first_synset = hyper[0]
                        else:
                            break_loop=False
                            steps=-2
                    else:
                        hyper = second_synset.hypernyms()
                        if (len(hyper) > 0):
                            second_synset = hyper[0]
                        else:
                            break_loop = False
                            steps = -2
                    steps+=1
                #Se è stato trovato un cammino
                if(steps>=0):
                    #Normalizzo per il valore massimo e moltiplico per 10 per avere un valore nel range [1,10]
                    index=normalize((-1*math.log((steps+1)/((2*max_depth)+1))),0,math.log(2*max_depth+1))*10
                else:
                    index=-1
                #Index rappresenta la similarità, in caso sia il nuovo massimo lo aggiorno
                if(index>max):
                    max=index
        #I massimi per ogni coppia vengono inseriti nella quinta cella in modo da creare un dataset con tutte le similarità
        row[5]=max
    return dataset

# Main e calcolo similarità e indici

In [6]:
#Carico il dataset
dataset=read_csv("WordSim353/WordSim353/WordSim353.csv")

#Calcolo similarità
dataset=calculate_wu_palmer(dataset)
dataset=calculate_shortest_path(dataset)
dataset=calculate_leakcock_chodorow(dataset)

#Estraggo i dati per maggiore leggibilità
human_coefficents=[row[2] for row in dataset]
wu_coefficents=[row[3] for row in dataset]
shortest_coefficents=[row[4] for row in dataset]
leak_coefficents=[row[5] for row in dataset]

#Calcolo coefficienti
correlation_coefficient,p_value = pearsonr(human_coefficents, wu_coefficents)
print("Perason for Wu Palmer: ",str(correlation_coefficient))
correlation_coefficient,p_value = spearmanr(human_coefficents, wu_coefficents)
print("Spearman rank for Wu Palmer: ",str(correlation_coefficient),"\n")

correlation_coefficient,p_value = pearsonr(human_coefficents, shortest_coefficents)
print("Perason for Shortest Path: ", str(correlation_coefficient))
correlation_coefficient,p_value = spearmanr(human_coefficents, shortest_coefficents)
print("Spearman rank for Shortest Path: ",str(correlation_coefficient),"\n")


correlation_coefficient,p_value = pearsonr(human_coefficents, leak_coefficents)
print("Perason for Leakcock & Chodorow: ", str(correlation_coefficient))
correlation_coefficient,p_value = spearmanr(human_coefficents, leak_coefficents)
print("Spearman rank for Leakcock & Chodorow: ",str(correlation_coefficient),"\n")

Perason for Wu Palmer:  0.20150625818011983
Spearman rank for Wu Palmer:  0.21943244102990547 

Perason for Shortest Path:  -0.022333216754054395
Spearman rank for Shortest Path:  0.16174828058822424 

Perason for Leakcock & Chodorow:  0.10251688106296773
Spearman rank for Leakcock & Chodorow:  0.16174828058822424 

